## Assigment 2: Word2vec and CBOW ⚡


In this assignment, we will guide you through the process of implementing Continous Bag Of Words (CBOW) using PyTorch, providing step-by-step instructions, code snippets, and explanations to aid your understanding. By the end of this assignment, you will have a solid grasp of Word2Vec's mechanics and be equipped to apply this knowledge to a wide range of NLP tasks.

Let's embark on this journey to unlock the power of word embeddings with Word2Vec and PyTorch!

This [guide](https://pytorch.org/tutorials/beginner/basics/intro.html) might come in handy to understand how Pytorch works.

Let's start by installing the needed libraries

In [ ]:
!gdown https://drive.google.com/file/d/1-mB6idLW5Jg4aE68jOj5NDcDxRNlMXpu/view?usp=sharing --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1-mB6idLW5Jg4aE68jOj5NDcDxRNlMXpu
To: /content/TestSet_sample.csv
100% 51.5k/51.5k [00:00<00:00, 66.7MB/s]


In [ ]:
book_urls = [
    "https://www.gutenberg.org/files/2600/2600-0.txt",
    "https://www.gutenberg.org/files/1342/1342-0.txt",
    "https://www.gutenberg.org/files/37106/37106-0.txt",
    "https://www.gutenberg.org/files/35461/35461-0.txt",
    "https://www.gutenberg.org/files/84/84-0.txt",
    "https://www.gutenberg.org/files/1513/1513-0.txt",
    "https://www.gutenberg.org/files/2701/2701-0.txt",
    "https://www.gutenberg.org/files/145/145-0.txt",
    "https://www.gutenberg.org/files/100/100-0.txt",
    "https://www.gutenberg.org/files/2641/2641-0.txt",
    "https://www.gutenberg.org/files/67979/67979-0.txt",
    "https://www.gutenberg.org/files/64317/64317-0.txt",
    "https://www.gutenberg.org/files/1080/1080-0.txt",
    "https://www.gutenberg.org/files/1184/1184-0.txt",
    "https://www.gutenberg.org/files/12342/12342-0.txt",
]

In this assigment we will use the data set wikitext. This dataset is composed of textual content extracted from Wikipedia articles which hopefully will lead to good embeddings when input in CBOW.

In [ ]:
import torch
torch.manual_seed(0)

As you can see there are a lot of numeric values, end-of-lines and the word "References" at the end. We saw in the previous assigment how preprocessing can significantly reduce the vocabulary; feel free to base the following code in your previous work.

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
import re
from nltk.corpus import stopwords
def preprocess_data(data):
  """ Method to clean text from noise and standarize text across the different classes.
      The preprocessing includes converting to joining all datapoints, lowercase, removing punctuation, and removing stopwords.
  Arguments
  ---------
  text : List of String
     Text to clean
  Returns
  -------
  text : String
      Cleaned and joined text
  """

  stop_words = set(stopwords.words('english'))

  text = ' '.join(data) #join all text in one single string
  text = text.lower() #make everything lower case
  text = text.strip() #remove \n characters
  text = text.replace("references", "") #remove word "References"
  text = re.sub(r'[^\w\s]', ' ', text) #remove any punctuation or special characters
  text = re.sub(r'\d+', ' ', text) #remove all numbers
  text = ' '.join([w for w in text.split() if w not in stop_words]) #remove all stopwords (see imports to help you with this)

  return text

In [ ]:
def read_files(file_paths):
    combined_text = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.readlines()
            pre_text = preprocess_data(data)
            combined_text.append(pre_text)
    return ' '.join(combined_text)

book_files = []
for i, url in enumerate(book_urls):
    filename = f"book_{i+1}.txt"
    !wget {url} -O {filename}
    book_files.append(filename)

text = read_files(book_files)

--2024-10-27 18:47:23--  https://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359405 (3.2M) [text/plain]
Saving to: ‘book_1.txt’

book_1.txt          100%[===================>]   3.20M  6.24MB/s    in 0.5s    

2024-10-27 18:47:24 (6.24 MB/s) - ‘book_1.txt’ saved [3359405/3359405]

--2024-10-27 18:47:24--  https://www.gutenberg.org/files/1342/1342-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752583 (735K) [text/plain]
Saving to: ‘book_2.txt’

book_2.txt          100%[===================>] 734.94K  2.03MB/s    in 0.4s    

2024-10-27 18:47:25 (2.03 MB/s) - ‘book

Our `text` variable includes all the data in just one string. Let's create a dictionary of frequencies for more pre-processing (same as in the last assigment).

In [ ]:
def vocab_frequency(text):
  """ Creates dictionary of frequencies based on a dataset.
  Arguments
  ---------
  dataset : list of tuples
      list of tuples of the form (label, text)
  Returns
  -------
  vocab_dict : dictonary
      Dictionary of words and their frequencies with the format {word: frequency}
  """
  vocab = {} #create empty dictionary

  #iterate through rows of df and count the frequency of words
  for word in text.split():
    if word not in vocab:
      vocab[word] = 1
    else:
      vocab[word] += 1

  return vocab

vocabulary = vocab_frequency(text)

In [ ]:
len(vocabulary)

68994

Now our vocabulary is ready 😎

Since our neural networks can't understand strings we have to turn these strings into integers. In this case, we can use the vocabulary index to represent the words, so we will need the mapping: word -> index

In [ ]:
import torch
import torch.nn as nn

In [ ]:
def word_to_indexer(vocabulary):
  """ Method to create vocabulary to index mapping.
  Arguments
  ---------
  vocabulary : Dictionary
     Dictonary of format {word:frequency}
  Returns
  -------
  word_to_index : Dictionary
      Dictionary mapping words to index with format {word:index}
  """
  word_to_index = {}
  word_to_index['OOV!'] = 0  #Create key,value pair for out of vocabulary words

  for index, (key, _) in enumerate(vocabulary.items()):
    word_to_index[key] = index + 1

  return word_to_index

word_to_index = word_to_indexer(vocabulary)

Let's start to get more on topic. If you remember, Word2Vec has a specific way of generating each vector.
Captura de pantalla 2024-05-21 161613.jpg

The embedding is based on the surrounding words (before and after) taken as context. In the case of CBOW, we input the surrounding words to our neural network and expect that it predicts the middle word. Let's format our dataset to meet the needs of the neural network.


In [ ]:
def generate_dataset(data, window_size, word_to_index):
  """ Method to generate training dataset for CBOW.
  Arguments
  ---------
  data : String
     Training dataset
  window_size : int
     Size of the context window
  word_to_index : Dictionary
     Dictionary mapping words to index with format {word:index}
  Returns
  -------
  surroundings : N x W Tensor
      Tensor with index of surrounding words, with N being the number of samples and W being the window size
  targets : Tensor
      Tensor with index of target word
  """
  surroundings= []
  targets = []
  data = data.split(" ")
  #TODO complete function
  for i in range(window_size,len(data)-window_size):
    #get surrounding words based on window size
    surrounding = [word_to_index[data[j]] for j in range(i - window_size, i)] + [word_to_index[data[j]] for j in range(i + 1, i + window_size + 1)]
    target = word_to_index[data[i]] #get target word (middle word)
    surroundings.append(surrounding) #append to surrounding
    targets.append(target) #append to targets
  surroundings = torch.tensor(surroundings)
  targets = torch.tensor(targets)
  return surroundings, targets

t_surroundings, t_targets = generate_dataset(text,2,word_to_index)

With our dataset ready we can finally create our Neural Network. The idea is to replicate what Mikolov did in 2013 (see slides of Word2Vec).

In [ ]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, embed_dim=300):
    """ Class to define the CBOW model
    Attributes
    ---------
    device : device
      Device where the model will be trained (gpu preferably)
    vocab_size : int
      Size of the vocabulary
    embed_dim : int
      Size of the embedding layer
    hidden_dim : int
      Size of the hidden layer
    """
    super().__init__()
    self.embedding = self.embedding = nn.Embedding(vocab_size, embed_dim) #use this layer to get a vector from the the word index
    self.linear = nn.Linear(embed_dim, vocab_size) #first fully connected layer (bottleneck)

  def forward(self, x):
    emb = self.embedding(x) #pass input through embedding layer
    average = torch.mean(emb, dim=1) #average and resize (size must be batch_size x embed_dim)
    out = self.linear(average) #pass through linear layer

    return out

In [ ]:
from torch.utils.data import DataLoader
#creation of dataloader for training
train_dataloader=DataLoader(list(zip(t_surroundings,t_targets)),batch_size=50,shuffle=True) #Here please change batch size depending of your GPU capacities (if GPU runs out of memory lower batch_size)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #CAUTION: RUN THIS CODE WITH GPU, CPU WILL TAKE TOO LONG
device

device(type='cuda')

In [ ]:
from tqdm import tqdm
model = CBOW(len(word_to_index)).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 10

#BE PATIENT: This code can take up to 2 hours and 10 min for a batch size of 64 and 10 epochs
for epoch in range(epochs):
    total_loss = 0
    for surr, tar in tqdm(train_dataloader):
        surr, tar = surr.to(device), tar.to(device)
        optimizer.zero_grad()

        log_probs = model(surr)

        loss= loss_function(log_probs,tar)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} loss: {total_loss/len(train_dataloader)}")

100%|██████████| 41486/41486 [11:08<00:00, 62.08it/s]


Epoch 1 loss: 8.5780451369712


100%|██████████| 41486/41486 [11:07<00:00, 62.16it/s]


Epoch 2 loss: 7.794211158249927


100%|██████████| 41486/41486 [11:06<00:00, 62.23it/s]


...

Epoch 8 loss: 5.9016796170771535


100%|██████████| 41486/41486 [11:05<00:00, 62.29it/s]


Epoch 9 loss: 5.765095515754134


100%|██████████| 41486/41486 [11:05<00:00, 62.29it/s]

Epoch 10 loss: 5.650529025439235


In [ ]:
torch.save(model.state_dict(), '/content/cbow_10_epoch_books.pt')

## Let's test it! 🤓

In [ ]:
model = CBOW(len(word_to_index))
model.load_state_dict(torch.load('/content/cbow_10_epoch_books.pt', map_location=torch.device('cpu')))

#Set the model to evaluation mode as it turns off dropout layers and similar training-only features
model.eval()

<ipython-input-17-c6591ee16c5b>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/cbow_10_epoch_books.pt', map_location=torch.devic

CBOW(
  (embedding): Embedding(68995, 300)
  (linear): Linear(in_features=300, out_features=68995, bias=True)
)

Now that we hopefully have good embeddings we can put them to the test. Let's start by creating some useful functions.

In [ ]:
def get_embedding(word, model, word_to_index):
  """ Method to get the embedding vector for a given word.
  Arguments
  ---------
  word : String
     Word given
  model : NN.module
     CBOW model
  word_to_index : Dictionary
     Dictionary mapping words to index with format {word:index}
  Returns
  -------
  word_embedding : Tensor
      Embedding vector for the given word
    """
  index = word_to_index.get(word, 0) # get word index

  with torch.no_grad():
        embedding_weights = model.embedding.weight # Get the weights of the embedding layer
        embedding_weights.requires_grad = False
        word_embedding = embedding_weights[index] # Extract the embedding vector for the given word index

  return word_embedding

get_embedding("shot",model,word_to_index) #test this code by getting embedding of the word "shot"

tensor([-2.0982e+00, -6.2335e-01,  1.6281e-01, -3.0909e+00, -2.2534e+00,
        -7.5596e-02,  7.7278e-01,  1.2902e+00, -4.4959e+00,  1.2315e+00,
         3.3595e-01,  1.7849e-01,  1.3308e+00,  3.0238e-01,  4.0420e-01,
        -1.3245e+00,  3.4329e+00, -8.7282e-01, -1.8569e+00, -2.9438e-01,
         1.5626e-01, -5.5852e-01,  1.5145e+00,  2.0709e+00, -4.1295e-01,
         1.3534e+00, -7.8593e-01,  3.7741e-01, -6.4420e-01, -4.2796e-01,
        -2.6704e-01,  1.3805e+00, -1.4233e+00,  1.6896e+00, -2.2131e-01,
         3.2451e-01,  8.0426e-01, -8.8517e-01, -1.3616e+00,  5.2892e-01,
        -7.3432e-01, -1.6934e+00,  4.1030e-01,  1.2755e+00, -2.8940e+00,
        -1.3345e+00, -2.2977e+00, -1.1434e+00, -1.1915e+00, -4.5858e-01,
         8.5228e-01,  1.2845e-01, -3.2577e-01,  1.8764e+00,  3.8836e-01,
         4.6290e-01,  3.0981e+00, -1.8829e+00,  1.7350e-02, -5.9335e-01,
         1.8626e+00,  3.5005e-01,  9.6240e-01,  2.6128e+00,  1.9523e+00,
        -4.9432e-01,  2.9974e+00,  4.0883e-01, -5.1

In [ ]:
def cosine_similarity(v1,v2):
  """ Method to calculate cosine similarity between two vectors.
  Arguments
  ---------
  v1 : Tensor
     First vector
  v2 : Tensor
     Second vector
  Returns
  -------
  cosine_similarity : float
      Cosine similarity between v1 and v2
  """
  return nn.functional.cosine_similarity(v1, v2, dim=0).item() #TODO create expression to calculate cosine similarity

In [ ]:
france = get_embedding("france",model,word_to_index)
paris = get_embedding("paris",model,word_to_index)
cosine_similarity(france,paris)

0.18748338520526886

In [ ]:
cosine_similarity(paris,paris)

1.0

In [ ]:
a = get_embedding("capital",model,word_to_index)
cosine_similarity(a,paris)

0.16332678496837616

In [ ]:
def get_k_nearest_words(k, word, vocabulary, model, word_to_index):
  """ Method to find the k nearest words of a given vector
  Arguments
  ---------
  k : int
     Number of nearest words to return
  word : torch.Tensor
     Embedding vector for the given word
  vocabulary : Dictionary
     Dictionary mapping words to frequency with format {word:frequency}
  model : NN.module
     CBOW model
  word_to_index : Dictionary
     Dictionary mapping words to index with format {word:index}
  Returns
  -------
  similar : List of Strings
      List of k nearest words to the given word
  """
  similarity_scores= torch.zeros(len(word_to_index))

  #fill similarity scores matrix using the word and our cosine_similarity function
  for v_word, v_index in word_to_index.items():
    if v_index == 0:
        continue
    v_word_embedding = get_embedding(v_word, model, word_to_index)
    similarity_scores[v_index] = cosine_similarity(word, v_word_embedding)

  k_first= torch.topk(similarity_scores,k).indices
  similar = [list(vocabulary.keys())[i-1] for i in k_first]

  return similar

The Google analogy test set is a state of the art data set for word embeddings that tests their ability to solve analogy tasks. A well-known example for it is: "king is to man as queen is to women." We will use a subset of this dataset to see how our CBOW model performs.


In [ ]:
import pandas as pd

def test_analogy(model, word_to_index, analogy_file, category = 'capital-common-countries'):
  """ Method to test accuracy of CBOW embeddings on analogy tasks.
  Arguments
  ---------
  mode : nn.Module
     CBOW model
  word_to_index : Dictionary
     Dictionary mapping words to index with format {word:index}
  analogy_file : String
     File containing analogy tasks
  Returns
  -------
  accuracy : float
      accuracy of the model on the analogy tasks
  """
  df = pd.read_csv(analogy_file)
  df = df[df.category==category] #using capital cities subset of test set
  correct = 0
  total = 0
  for index,row in df.iterrows():
      word_one = row['word_one'].lower()
      word_two = row['word_two'].lower()
      word_three = row['word_three'].lower()
      word_four = row['word_four'].lower()
      #remember to standarize the words by using .lower

      #create exception in case word is not in vocabulary
      if (word_one not in word_to_index or word_two not in word_to_index or
          word_three not in word_to_index or word_four not in word_to_index):
        continue

      #get embedding of all words
      emb_one = get_embedding(word_one, model, word_to_index)
      emb_two = get_embedding(word_two, model, word_to_index)
      emb_three = get_embedding(word_three, model, word_to_index)

      result = emb_two - emb_one + emb_three #calculate embedding_word_2-embedding_word_1+embedding_word_3
      prediction = get_k_nearest_words(10, result, vocabulary, model, word_to_index) #call the k_nearest function you created before (set k to 10)

      # if word_four is in prediction, add one to correct
      if word_four in prediction:
        # print(f'found {word_four} = {word_two} - {word_one} + {word_three}')
        correct += 1

      total+=1
  if total != 0:
    accuracy=correct/total
  else:
    return 'No word was found in the embeddings '
  return accuracy

In [ ]:
test_analogy(model,word_to_index,'TestSet_sample.csv')

0.06043956043956044

In [ ]:
test_analogy(model,word_to_index,'TestSet_sample.csv', 'family')

0.1323529411764706

Our model can do much better! In the "Let's experiment" part we will try to improve this model. Also depending on your GPU accesibility you can train for more epochs and data (making trim_dataset bigger in cell #3).

TSNE can be a very helpful and cool visualization method of our vectors, this way we can clearly see the relationship between different words (maybe even find some clusters!). Let's implement this visualization.

In [ ]:
import numpy as np
import pandas as pd
import torch
import sys

from sklearn.manifold import TSNE
import plotly.graph_objects as go

tsne = TSNE() #create TSNE
embeddings = model.embedding.weight[:1000].detach().cpu().numpy() #get embeddings by detaching weights of embedding layer (use only first 1000 words)
#TODO create TSNE plot to visualize the relationship between our vectors
tsne_result = tsne.fit_transform(embeddings)
words = list(word_to_index.keys())[:1000]
fig = go.Figure(data=go.Scatter(
    x=tsne_result[:, 0],
    y=tsne_result[:, 1],
    mode='markers',
    text=words,
    hoverinfo='text'
))
fig.show()

Since the TSNE isn't shown when the notebook is downloaded, here is the link to the colab notebook: https://colab.research.google.com/drive/1XYeae49NiC2tTs9r7uO_5RMj8aqw5Hw9?usp=sharing \\
p.s: you can hover your mouse over each point to see the word